In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [ ]:
# Target 열에 예측해야 하는 출력값을 넣어준다.
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns = california.feature_names)
df["Target"] = california.target

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
'''
캘리포니아 주택 데이터셋의 값을 파이토치 Float 텐서로 변환한다.
입력 데이터를 x 변수에 슬라이싱하여 할당하고, 출력 데이터를 y 변수에 슬라이싱하여 할당한다.
'''
data = torch.from_numpy(df.values).float()

x = data[:, :-1]
y = data[:, -1:]

print(x.size(), y.size())

In [ ]:
'''
준비된 입출력을 임의의 학습, 검증, 테스트 데이터로 나눈다.
60%의 학습 데이터와, 20%의 검증 데이터, 20%의 테스트 데이터로 구성하기 위해 미리 비율을 설정한다.
'''

# Train, Valid, Test ratio
ratios = [.6, .2, .2]

# radios에 담긴 값을 활용하여, 실제 데이터셋에서 몇 개의 샘플들이 각각 학습, 검증, 테스트셋에 할당되어야 하는지 구할 수 있다.
train_cnt = int(data.size(0) * ratios[0])
valid_cnt = int(data.size(0) * ratios[1])
test_cnt = data.size(0) - train_cnt - valid_cnt
cnts = [train_cnt, valid_cnt, test_cnt]

# 전체 20640개의 샘플 중, 학습 데이터는 12384개, 검증 데이터와 테스트 데이터는 4128개 이다.
print("Train %d | valie %d | Test %d samples" % (train_cnt, valid_cnt, test_cnt))


In [ ]:
'''
위의 과정에서 중요한 점은, 앞에서 설정한 비율로 데이터셋을 나누되 임의로 샘플을 설정하여 진행해야 한다는 것이다.
다음 코드를 통해, 데이터셋의 랜덤 셔플링 후 나누기를 수행한다.
주목해야할 점은 x와 y에 대해서, 각각 랜덤 선택 작업을 수행하는 것이 아니라, 쌍으로 짝지어 수행이 된다는 것이다.
- 만일, x와 y를 각각 따로 섞어서 둘의 관계가 깨져버린다면, 아무 의미 없는 노이즈로 가득한 데이터가 될 것이다.
'''

# Shuffle before split
indices = torch.randperm(data.size(0))

x = torch.index_select(x, dim=0, index=indices)
y = torch.index_select(y, dim=0, index=indices)

# Split train, valid and test with each count
x = list(x.split(cnts, dim=0))
y = y.split(cnts, dim=0)

for x_i, y_i in zip(x, y):
    print(x_i.size(), y_i.size())
'''
torch.Size([12384, 8]) torch.Size([12384, 1])
torch.Size([4128, 8]) torch.Size([4128, 1])
torch.Size([4128, 8]) torch.Size([4128, 1])
'''

In [ ]:
'''
위의 작업이 끝나면, 6:2:2의 비율대로 학습, 검증, 테스트셋이 나눠진 것을 확인할 수 있다.
이제 학습에 들어가면, 매 에포크마다 12384개의 학습 샘플들을 임의로 섞어 미니배치들을 구성하여, 학습 이터레이션을 돌게 된다.

데이터셋이 정상적으로 나뉜 것을 확인했다면, 데이터셋 정규화 작업도 수행한다.
- 표준 스케일링을 진행하기 위해서는 데이터셋의 각 열에 대한 평균과 표준편차를 구해야 한다.
- 이 과정을 통해, 데이터셋의 각 열의 분포를 추정하고, 추정된 평균과 표준편차를 활용하여 표준정규분포로 변환한다.
- 만일, 검증 데이터 또는 테스트 데이터를 학습 데이터와 합친 상태에서 평균과 표준편차를 추정한다면, 정답을 보는 것과 동일하다.
    - 아주 잘 정의된 매우 큰 데이터셋이라면 임의로 학습 데이터와 검증/테스트 데이터셋을 나눌 때 그 분포가 매우 유사할 수 있다.
        - 그렇다고 해서 학습 데이터와 테스트 데이터를 합친 상태에서 평균과 표준편차를 추정해도 되는 것은 아니다.

아래의 코드는 학습 데이터인 x[0]에 대해서 표준 스케일링을 피팅시키고, 이후에 해당 스케일러를 활용하여 학습(x[0]), 검증(x[1]), 테스트(x[2]) 데이터에 대해 정규화를 진행한다.
- 이처럼 학습 데이터만을 활용하여 정규화를 정규화를 진행하는 것은 매우 중요하다.
'''
scaler = StandardScaler()
scaler.fit(x[0].numpy()) # fit with train data only

x[0] = torch.from_numpy(scaler.transform(x[0].numpy())).float()
x[1] = torch.from_numpy(scaler.transform(x[1].numpy())).float()
x[2] = torch.from_numpy(scaler.transform(x[2].numpy())).float()

In [ ]:
# 학습 코드 구현
# nn.Sequential을 활용하여 모델을 선언하고, 아담 옵티마이저에 등록한다.
model = nn.Sequential(
    nn.Linear(x[0].size(-1), 6), # size(-1)은 텐서의 마지막 차원을 가리킨다.
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 4),
    nn.LeakyReLU(),
    nn.Linear(4, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y[0].size(-1)),
)

optimizer = optim.Adam(model.parameters())

In [ ]:
# 모델 학습에 필요한 세팅 값을 설정한다.
n_epochs = 4000
batch_size = 256
print_interval = 100

In [ ]:
'''
앞서, 우리가 원하는 모델은 가장 낮은 검증 손실 값을 갖는 모델이다.
- 즉, 매 에포크마다 학습이 끝날 때 검증 데이터셋을 똑같이 피드포워딩하여, 검증 데이터셋 전체에 대한 평균 손실 값을 구하고, 이전 에포크의 검증 손실 값과 비교하는 작업을 수행해야 한다.
    - 만일, 현재 에포크의 검증 손실 값이 이전 에포크 까지의 최저 검증 손실 값보다 더 낮다면, 최저 검증 손실 값은 새롭게 갱신되고 현재 에포크의 모델은 따로 저장되어야 한다.
    - 현재 에포크의 검증 손실 값이 이전 에포크까지의 최저 검증 손실 값보다 크다면, 이번 에포크의 모델을 따로 저장할 필요 없이 넘어가면 된다.

학습이 모두 끝났을 때, 정해진 에포크가 n_epochs번 진행되는 동안 최저 검증 손실값을 뱉어냈던 모델이 우리가 원하는 일반화가 잘된 모델이라고 볼 수 있다.
- 즉, 학습이 종료되고 나면, 최저 검증 손실 값의 모델을 다시 복원하고 사용자에게 반환하면 된다.

아래의 코드는 최저 검증 손실을 추적하기 위한 lowest_loss와 최저 검증 손실 값을 뱉어낸 모델을 저장하기 위한 변수 best_model을 미리 생성하는 모습을 보여주고 있다.
- 이때, best_model에 단순히 현재 모델을 저장한다면, 얇은 복사가 수해오디어 주소값이 저장되므로, 깊은 복사를 통해 값 자체를 복사하여 저장해야 한다.
    - 깊은 복사를 위해, copy 패키지의 deepcopy 함수를 사용한다.
또한, 학습 조기 종료를 위한 세팅 값과 가장 낮은 검증 손실 값을 뱉어낸 에포크를 저장하기 위한 변수인 lowset_epoch도 선언한다.

이후, 학습을 위한 for문을 수행한다.
이전까지의 코드와 달라진 점은, 바깥쪽 for 문 후반부에 검증 작업을 위한 코드가 추가되었다는 것이다.
- 새롭게 추가된 코드의 내용운, 학습/검증 손실 값 히스토리를 저장하기 위한 train_history와 valid_history 변수가 추가되었다는 것이다.
'''

from copy import deepcopy

lowest_loss = np.inf
best_model = None

early_stop = 100
lowest_epoch = np.inf

train_history, valid_history = [], []

for i in range(n_epochs):
    # Shuffle before mini-batch split.
    indices = torch.randperm(x[0].size(0))
    x_ = torch.index_select(x[0], dim=0, index = indices)
    y_ = torch.index_select(y[0], dim=0, index=indices)
    # |x_| = (total_size, input_dim)
    # |y_| = (total_size, output_dim)

    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)
    # |x_[i]| = (batch_size, input_dim)
    # |y_[i]| = (batch_size, output_dim)

    train_loss, valid_loss = 0, 0
    y_hat = []

    for x_i, y_i in zip(x_, y_):
        # |x_i| = |x_[i]|
        # |y_i| = |y_[i]|
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        train_loss += float(loss)
    train_loss = train_loss / len(x_)

    '''
    이처럼 학습 데이터셋을 미니배치로 나눠 한 바퀴 학습하고나면, 검증 데이터셋을 활용하여 검증 작업을 수행한다.
    학습과 달리 검증 작업은 역전파를 활용하여 학습을 수행하지 않습니다.
    - 즉, 그래디언트를 계산할 필요가 없기에, torch.no_grad 함수를 호출하여, with 내부에서 검증 작업을 진행한다.
        - 그래디언트를 계산하기 위한 배후 작업들이 없어지기 때문에, 계산 오버헤드가 줄어들고 속도가 빨라지고 메모리 사용량도 줄어들게 된다.

    with 내부를 살펴본다.
    - split 함수를 사용하여, 미니배치크기로 나눠 주는 것을 볼 수 있다.
    - 앞에서 설명한 것처럼 검증 작업은 메모리 사용량이 적기 때문에 검증 작업을 위한 미니배치 크기는 학습용보다 더 크게 가져가도 되지만, 간편함을 위해 기존 학습용 미니배치 크기와 같은 크기를 사용한다.
    - 또한, 학습과 달리 셔플링 작업이 빠진 것을 볼 수 있다.
    - 또한, for 반복문 내부에도 피드포워드만 있고, 역전파 관련 코드는 없다.
    '''

    # You need to declare to PYTORCH to stop build the computation graph
    with torch.no_grad():
        # You dont need to shuffle the validation set
        # Only split is needed
        x_ = x[1].split(batch_size, dim=0)
        y_ = y[1].split(batch_size, dim=0)

        valid_loss = 0

        for x_i, y_i in zip(x_, y_):
            y_hat_i = model(x_i)
            loss = F.mse_loss(y_hat_i, y_i)

            valid_loss += loss

            y_hat += [y_hat_i]
    valid_loss = valid_loss / len(x_)

    # Log each loss to plot after training is done
    train_history += [train_loss]
    valid_history += [valid_loss]

    if(i + 1) % print_interval == 0:
        print("Epoch %d : train loss=%.4e valid_loss=%.4e lowest_loss=%.4e" %(i+1, train_loss, valid_loss, lowest_loss))
    
    '''
    앞과 같이 학습과 검증 작업이 끝나고 나면, 검증 손실 값을 기준으로 모델 저장 여부를 결정한다.
    우리가 원하는 것은 검증 손실을 낮추는 것이다.
    - 즉, 기존 최소 손실 값 변수 lowest_loss와 현재 검증 손실 값 valid_loss를 비교하여, 최소 손실 값이 갱신될 경우 현재 에포크의 모델을 저장한다.
    - 또한, 정해진 기간(early_stop) 변수 동안 최소 검증 손실 값의 갱신이 없을 경우, 학습을 종료한다.
    조기 종료 파라미터 또한, 하이퍼파라미터이다.
    '''
    if valid_loss <= lowest_loss:
        lowest_loss = valid_loss
        lowest_epoch = i

        # 'state_dict()' returns model weights as key-value
        # Take a deep copy, if the valid loss is lowest ever
        best_model = deepcopy(model.state_dict())
    else:
        if early_stop > 0 and lowest_epoch + early_stop < i + 1:
            print("There is no improvement during last %d epochs." % early_stop)
            break
print("The best validation loss from epoch %d: %.4e" % (lowest_epoch + 1, lowest_loss))
# Load best epoch's model
model.load_state_dict(best_model)

In [ ]:
'''
state_dict 함수가 현재 모델 파라미터를 key-value 형태의 주소값으로 0을 반환하기에, 그냥 변수에 state_dict 겨로가값을 지정할 경우, 에포크가 끝날 때마다 best_model에 저장된 값이 변경될 수 있다.
- 즉, deepcopy를 활용하여 현재 모델의 가중치 파라미터를 복사하여 저장한다.

모든 작업이 수행되고 나면 for문을 빠져나와 best_model에 저장되어 있던 가중치 파라미터를 모델 가중치 파라미터로 복원한다.
그러면, 최소 검증 손실 값을 얻은 모델로 되돌릴 수 있게 된다.

코드를 수행하면 다음과 같이 출력되는 것을 볼 수 있다.
29번째 에포크에서 최소 검증 손실 값을 얻었음을 알 수 있다.
- 만일 손실 값이 좀 더 떨어질 여지가 있다면, 조기 종료 파라미터를 늘릴 수도 있다.
'''

In [ ]:
'''
손실 곡선 확인

이제 train_history, valid_history에 쌓인 손실 값을 그림으로 그려서 확인한다.
이렇게, 그림을 통해서 확인하면 화면에 프린트된 숫자들을 보다 훨씬 쉽게 손실 값 추세를 확인할 수 있다.
'''
plot_from = 10

plt.figure(figsize=(20, 10))
plt.grid(True)
plt.title("Train / Valid Loss History")
plt.plot(
    range(plot_from, len(train_history)), train_history[plot_from:],
    range(plot_from, len(valid_history)), valid_history[plot_from:],
)
plt.yscale("log")
plt.show()

In [ ]:
'''
눈에 띄는 부분은, 대부분의 구간에서 검증 손실 값이 학습 손실 값 보다 낮다는 것이다.
- 검증 데이터셋은 학습 데이터셋에 비해, 일부에 불과하기 때문에 편향이 있을 수 있다.
- 따라서 우연히 검증 데이터셋이 좀 더 쉽게 구성이 되었다면 학습 데이터셋에 비해 더 낮은 손실 값을 가질 수도 있다.
- 만일 이 두 손실 값이 너무 크게 차이가 나지 않는다면 크게 신경 쓰지 않아도 된다.

또한, 검증 손실 값과 학습 손실 값의 차이가 학습 후반부로 갈수록 점점 줄어드는 것을 확인할 수 있다.
모델이 학습 데이터에만 존재하는 특성을 학습하는 과정이라고 볼 수 있다.
하지만, 검증 손실 값도 천천히 감소하고 있는 상황이므로, 온전히 오버피팅에 접어들었다고 볼 수 없다.
- 조기 종료를 하지 않고, 계속 학습시킨다면 오버피팅이 될 것이다.
- 조기 종료 파라미터 등을 바꿔 가며 좀 더 낮은 검증 손실 값을 얻기 위한 튜닝을 할 수 있다.
'''

In [ ]:
# 결과 확인
'''
테스트 데이터셋에 대해서도 성능을 확인한다.
우리의 최종 목표는 테스트 성능이 좋은 모델을 얻는 것이지만, 이 과정에서 학습 데이터셋과 검증 데이터셋만 활용할 수 있었고, 중간 목표는 검증 손실 값을 낮추는 것이였다.
이제 이렇게 얻어진 모델이 테스트 데이터셋에 대해서도 여전히 좋은 성능을 유지하는지 확인한다.

아래의 코드를 보면 검증 작업과 거의 비슷하게 진행되는 것을 알 수 있다.
torch.no_grad 함수를 활용하여 with 내부에서 그래디언트 계산 없이 모든 작업이 수행된다.
또한 미니배치 크기로 분할하여 for 반복문을 통해 피드포워드한다.
'''

test_loss = 0
y_hat = []

with torch.no_grad():
    x_ = x[2].split(batch_size, dim=0)
    y_ = y[2].split(batch_size, dim=0)

    for x_i, y_i in zip(x_, y_):
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        test_loss += loss # Gradient is already detached
        y_hat += [y_hat_i]
test_loss = test_loss / len(x_)
y_hat = torch.cat(y_hat, dim=0)

sorted_history = sorted(zip(train_history, valid_history), key=lambda x : x[1])

print("Train loss : %.4e" % sorted_history[0][0]) # Train loss : 2.9569e-01
print("Valid loss : %.4e" % sorted_history[0][1]) # Valid loss : 3.1284e-01
print("Test loss : %.4e" % test_loss) # Test loss : 3.3805e-01

In [ ]:
'''
마지막으로 sorted 함수를 활용하여 가장 낮은 검증 손실 값과 이에 대응하는 학습 손실 값을 찾아서 테스트 손실 값과 함께 출력한다.
최종적으로 이 모델은 0.33805이라는 테스트 손실 값이 나오는 것으로 확인되었다.
만일, 다른 방법론을 적용하거나 모델 구조 변경 등을 한다면 0,33805라는 테스트 손실 값이 나오는 모델이 베이스라인 모델이 될 것이다.
그렇게 되면, 새로운 모델은 이 베이스라인을 이겨야 된다.
- 정단한 비교를 위해 매번 랜덤하게 학습/검증/테스트셋을 나누기 보다 아예 테스트셋을 따로 빼두는 것도 방법이다.

샘플들에 대한 예측 값을 페어 플롯을 그린다.
- 테스트셋에 대해서만 플롯을 그린다.
'''
df = pd.DataFrame(torch.cat([y[2], y_hat], dim=1).detach().numpy(), columns=["y", "y_hat"])

sns.pairplot(df, height = 5)
plt.show()